# IRIS Flower classification

Step 1: Loading and preparing the data for training

In [43]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
x_norm = (X_train - np.mean(X_train)) / np.std(X_train)
x_train = torch.from_numpy(x_norm).float()
y_train = torch.from_numpy(y_train)

train_data = DataLoader(TensorDataset(x_train, y_train), batch_size=2)

for x, y in train_data:
    print(x)
    print(y)
    break

tensor([[ 0.5883,  0.0789, -1.2458, -1.6534],
        [ 1.1488,  0.4864, -0.9910, -1.5515]])
tensor([0, 0])


Step 2: Define the model architecture

In [44]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self,in_size,hid_size,out_size):
        super().__init__()
        self.layer1 = nn.Linear(in_size,hid_size)
        self.layer2 = nn.Linear(hid_size,out_size)

    def forward(self,x):
        x = self.layer1(x)
        x = nn.Sigmoid()(x)
        x = self.layer2(x)
        return nn.Softmax(dim=1)(x)

input_size = x_train.shape[1]
hidden_size = 16
output_size = 3

model = NeuralNetwork(input_size,hidden_size,output_size)    
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.01)


Step 3: Train the model

In [45]:
from matplotlib.pylab import f


epoches = 10
loss_hist = [0] * epoches
correct = [0] * epoches

for epoch in range(epoches):
    for x, y in train_data:

        output = model(x)
        loss = loss_fn(output, y)

        loss.backward()
        optim.step()
        optim.zero_grad()

        # calculate accuracy and loss
        loss_hist[epoch] += loss.item() * y.shape[0]
        is_correct = (torch.argmax(output, dim=1) == y).float()
        correct[epoch] += is_correct.sum().item()
    
    loss_hist[epoch] /= len(train_data)
    correct[epoch] /= len(train_data)

print(f"loss: {loss_hist}")
print(f"accuracy: {correct}")

loss: [2.125301929314931, 1.8207670172055563, 1.6377172271410625, 1.532715340455373, 1.4467649400234222, 1.3817280153433482, 1.335774431626002, 1.303670992453893, 1.2808169682820638, 1.264046130577723]
accuracy: [0.7666666666666667, 1.5, 1.6, 1.8666666666666667, 1.9166666666666667, 1.8833333333333333, 1.9166666666666667, 1.9166666666666667, 1.9166666666666667, 1.9166666666666667]


Step 4: Evaluate the model on the Test data

In [46]:
x_test = torch.from_numpy((X_test - np.mean(X_test)) / np.std(X_test)).float()
y_test = torch.from_numpy(y_test)

output = model(x_test)
loss = loss_fn(output, y_test)
print(f"loss: {loss}")

loss: 0.6167417168617249
